# Tool for Sending Logged-in Users a Reminder to Log Out for Maintenance

In [ ]:
%%capture
import sys
import os

!git clone https://github.com/ITISFoundation/osparc-simcore.git
    
# Weird call to pip as suggested by PCR via akevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/#How-to-use-Pip-from-the-Jupyter-Notebook
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install pytz
!{sys.executable} -m pip install psycopg2-binary
!{sys.executable} -m pip install secure-smtplib 

import datetime
import pytz
from pytz import timezone
zurichTZ = timezone('Europe/Zurich')
from dateutil.relativedelta import relativedelta

## Provide datetime of maintenance and excluded users

In [ ]:
# A list of user IDs you would like to send emails to - these can be found from redis commander
excluded_users = ["kuster@itis.swiss", "@itis.testing"] # can be only aprt of a mail address 

# Provide the time of maintenance
# datetime(year, month, day, hour, minute, second)
maintenance_time = datetime.datetime(2022, 8, 15, 17, 15, 0, tzinfo=zurichTZ)

productString = "o²S²PARC / S4L-Web / TI-Planning"


In [ ]:
deploymentString = os.environ.get('DEPLOYMENT_FQDNS')
# Compute timing inforation
utc = pytz.timezone('UTC')
now = utc.localize(datetime.datetime.utcnow())
attrs = ['years', 'months', 'days', 'hours', 'minutes', 'seconds']
human_readable = lambda delta: ['%d %s' % (getattr(delta, attr), attr if getattr(delta, attr) > 1 else attr[:-1]) for attr in attrs if getattr(delta, attr)]
assert(maintenance_time > now)
timeDiff = human_readable(relativedelta(seconds=(int(maintenance_time.strftime('%s')) - int(now.strftime('%s')))))
maintenanceInStr = " ".join(timeDiff[:2])
print("Maintenance in: ",maintenanceInStr)
maintenance_timeStr = maintenance_time.strftime('%H:%M')
print("Maintenance at: ",maintenance_timeStr)
maintenance_dateStr = maintenance_time.strftime('%A %d. %B %Y')
print("Maintenance on: ",maintenance_dateStr)

## Access the Database to Find Names and Emails

In [ ]:
import sqlalchemy as db
import psycopg2
import os
import sys
import json
import importlib
from pathlib import Path
#########################################
#######################
PG_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
PG_ENDPOINT=os.environ.get('POSTGRES_ENDPOINT')
PG_DB=os.environ.get('POSTGRES_DB')
PG_USER=os.environ.get('POSTGRES_USER')
pgEngineURL= "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        user=PG_USER,
        password=PG_PASSWORD,
        database=PG_DB,
        host=PG_ENDPOINT.split(":")[0],
        port=int(PG_ENDPOINT.split(":")[1]),
    )
engine = db.create_engine(pgEngineURL)
connection = engine.connect()
metadata = db.MetaData()
user_table = db.Table('users', metadata, autoload=True, autoload_with=engine)

## Find These Users

In [ ]:
name_list = []
email_list = []

query = db.select([user_table.c.name, user_table.c.email])
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()

if result_set:
    for i in result_set:
        userIsExcluded = False
        for excluded_mail in excluded_users:
            if excluded_mail in i[1]:
                userIsExcluded = True
                break
        if not userIsExcluded:
            name_list.append(i[0])
            email_list.append(i[1])
else:
    print("Querry to DB failed")


## Log into Mail Server to Send Emails

In [ ]:
## =======  For debugging =======
if False:
    name_list = ["Dustin"]
    email_list = ["kaiser@itis.swiss"]

In [ ]:
SMTP_HOST = os.environ.get('SMTP_HOST')
SMTP_PORT = os.environ.get('SMTP_PORT')
SMTP_USERNAME = os.environ.get('SMTP_USERNAME')
SMTP_PASSWORD =  os.environ.get('SMTP_PASSWORD')

## Send a Nice Email to Everyone Logged In

In [ ]:
print("You are about to send emails to: ")
print('\n'.join(email_list))
print("\nPlease be sure before continuing...")

In [ ]:
import smtplib, ssl

context = ssl.create_default_context()
with smtplib.SMTP(SMTP_HOST, SMTP_PORT) as server:
    server.ehlo()  # Can be omitted
    server.starttls(context=context)
    server.ehlo()  # Can be omitted
    server.login(SMTP_USERNAME, SMTP_PASSWORD)
    for name, email in zip(name_list, email_list):
        message = """Subject: """ + productString + """ Maintenance Warning

\nHi """ + name + """, \n
You are registered with an account on: """ + str(deploymentString) +""".

Please be warned that we will be doing a routine maintenance on these platforms on """ + str(maintenance_dateStr) + """ at """ + str(maintenance_timeStr) + """, Europe/Zurich timezone. To keep your work safe, we recommend
that you close your studies and log out before the maintenance begins. Apologies for any inconvenience.

Cheers,
your friendly """ + productString + """ team"""

        server.sendmail(SMTP_USERNAME, email, message.encode('utf-8'))